<a href="https://colab.research.google.com/github/ucfilho/ANN/blob/master/Churn_SVM_set_27_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# this churn dataset is not the same present in kaggle
# I have a big problem to clean the data
# it is an easy task but was not easy to found: some rows in column TotalCharges
# is blanck values and it made all column TotalCharges as string to convert in float with 
# blanck as NA it was necessary to use 
# df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
# PS: this command doesn't work:
#      df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], downcast="float")

In [3]:
url='https://raw.githubusercontent.com/treselle-systems/customer_churn_analysis/master/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [4]:
df= pd.read_csv(url, sep=",")

In [5]:
print(df.head(3))

   customerID  gender  SeniorCitizen  ... MonthlyCharges TotalCharges  Churn
0  7590-VHVEG  Female              0  ...          29.85        29.85     No
1  5575-GNVDE    Male              0  ...          56.95       1889.5     No
2  3668-QPYBK    Male              0  ...          53.85       108.15    Yes

[3 rows x 21 columns]


In [6]:
Names=df.columns
print(Names)

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [8]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [9]:
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes


In [10]:
# I had problem with index keep index create incompatibibility
#df.reset_index(drop=True)
df.drop(["customerID"],axis=1,inplace = True)

In [11]:
df

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7039,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7040,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes


In [12]:
Numeric=['TotalCharges','MonthlyCharges','tenure']
Names=df.columns

for item in Names:
  codes, uniques = pd.factorize(df[item])
  Change=True
  for x in Numeric:
    if(item==x):
      Change=False
      print('======',Change)
  if(Change):
    print(Change)
    df[item]=codes.copy()

True
True
True
True
====== False
True
True
True
True
True
True
True
True
True
True
True
True
====== False
====== False
True


In [13]:
print(df.head())

   gender  SeniorCitizen  Partner  ...  MonthlyCharges  TotalCharges  Churn
0       0              0        0  ...           29.85         29.85      0
1       1              0        1  ...           56.95       1889.50      0
2       1              0        1  ...           53.85        108.15      1
3       1              0        1  ...           42.30       1840.75      0
4       0              0        1  ...           70.70        151.65      1

[5 rows x 20 columns]


In [14]:
scaler = MinMaxScaler()
#X=df.drop(['Churn'],axis=1) 
# if you drop here when you exclude NA the x and y will have different number of elements
X=df.copy()
rows,cols=X.shape
scaler = MinMaxScaler()
W=pd.DataFrame(scaler.fit_transform(X[Numeric]),columns=Numeric)

In [15]:
W

,TotalCharges,MonthlyCharges,tenure
0,0.001275,0.115423,0.013889
1,0.215867,0.385075,0.472222
2,0.010310,0.354229,0.027778
3,0.210241,0.239303,0.625000
4,0.015330,0.521891,0.027778
...,...,...,...
7038,0.227521,0.662189,0.333333
7039,0.847461,0.845274,1.000000
7040,0.037809,0.112935,0.152778
7041,0.033210,0.558706,0.055556


In [16]:
X.dropna(inplace = True)

In [17]:
X.shape

(7032, 20)

In [18]:
for i in Numeric:
  X[i]=W[i].copy()

In [19]:
X

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,0,0,0.013889,0,0,0,0,0,0,0,0,0,0,0,0,0.115423,0.001275,0
1,1,0,1,0,0.472222,1,1,0,1,1,1,0,0,0,1,1,1,0.385075,0.215867,0
2,1,0,1,0,0.027778,1,1,0,1,0,0,0,0,0,0,0,1,0.354229,0.010310,1
3,1,0,1,0,0.625000,0,0,0,1,1,1,1,0,0,1,1,2,0.239303,0.210241,0
4,0,0,1,0,0.027778,1,1,1,0,1,0,0,0,0,0,0,0,0.521891,0.015330,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,0,0,1,0.333333,1,2,0,1,1,1,1,1,1,1,0,1,0.662189,0.227521,0
7039,0,0,0,1,1.000000,1,2,1,0,0,1,0,1,1,1,0,3,0.845274,0.847461,0
7040,0,0,0,1,0.152778,0,0,0,1,1,0,0,0,0,0,0,0,0.112935,0.037809,0
7041,1,1,0,0,0.055556,1,2,1,0,1,0,0,0,0,0,0,1,0.558706,0.033210,1


In [20]:
y=X['Churn']
X=X.drop(['Churn'],axis=1)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [22]:
model =SVC(kernel='rbf', gamma=0.1,C=100,tol=1e-3)
model.fit(X_train, y_train)

SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [23]:
#Predicting new data
predictions = model.predict(X_test)

In [24]:
print(classification_report(y_test,predictions))
print('\n')
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

           0       0.84      0.85      0.84      1580
           1       0.54      0.52      0.53       530

    accuracy                           0.77      2110
   macro avg       0.69      0.68      0.69      2110
weighted avg       0.76      0.77      0.77      2110



[[1342  238]
 [ 255  275]]
